# Cell Migration Quantification

<div class="alert alert-info">
<h3 style="margin-top: 0;"> Instructions <i class="fa fa-info-circle"></i></h3>
Run each cell. More useful instructions will be added later.
</div>

In [5]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

# Requirements

## For developers

<div class="alert alert-warning">
<h3 style="margin-top: 0;"> Warning <i class="fa fa-exclamation-triangle"></i></h3>
LMFIT has been removed --> No need to install it anymore!
</div>

```
Python 3.6 Kernel, but no reason why this won't work on 3.7 if these libraries are present
Collecting lmfit==0.9.12
Collecting uncertainties>=3.0 (from lmfit==0.9.12)
Collecting scipy>=0.17 (from lmfit==0.9.12)
  Downloading https://files.pythonhosted.org/packages/7f/5f/c48860704092933bf1c4c1574a8de1ffd16bf4fde8bab190d747598844b2/scipy-1.2.1-cp36-cp36m-manylinux1_x86_64.whl (24.8MB)
    100% |████████████████████████████████| 24.8MB 236kB/s eta 0:00:01    68% |██████████████████████          | 17.0MB 41.9MB/s eta 0:00:01
Collecting six>1.10 (from lmfit==0.9.12)
  Downloading https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
Collecting asteval>=0.9.12 (from lmfit==0.9.12)
Collecting numpy>=1.10 (from lmfit==0.9.12)
  Downloading https://files.pythonhosted.org/packages/35/d5/4f8410ac303e690144f0a0603c4b8fd3b986feb2749c435f7cdbb288f17e/numpy-1.16.2-cp36-cp36m-manylinux1_x86_64.whl (17.3MB)
    100% |████████████████████████████████| 17.3MB 247kB/s eta 0:00:01
Installing collected packages: uncertainties, numpy, scipy, six, asteval, lmfit
  Found existing installation: numpy 1.14.0
    Uninstalling numpy-1.14.0:
      Successfully uninstalled numpy-1.14.0
  The scripts f2py, f2py3 and f2py3.6 are installed in '/home/jovyan/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Found existing installation: scipy 1.0.0
    Uninstalling scipy-1.0.0:
      Successfully uninstalled scipy-1.0.0
  Found existing installation: lmfit 0.9.12
    Uninstalling lmfit-0.9.12:
      Successfully uninstalled lmfit-0.9.12
Successfully installed asteval-0.9.13 lmfit-0.9.12 numpy-1.16.2 scipy-1.2.1 six-1.12.0 uncertainties-3.0.3
also:

opencv-python                     4.0.0.21
seaborn == 0.9.0
```

## Define some functions and import some others

In [41]:
%matplotlib inline
# from lmfit import Minimizer, Parameters, report_fit
import cv2
import numpy as np
from skimage import draw
from skimage import io
import matplotlib.pyplot as plt
from scipy import optimize
import humanfriendly
from timeit import default_timer as timer
import os
import pandas as pd
import seaborn as sns
from cuzcatlan import add_stat_annotation

In [16]:
def cost(params):
    global im2
    maxy, maxx = im2.shape
    maxr = min(maxx,maxy)/2
    area = maxy*maxx
    
    x0= params[0]
    y0 = params[1]
    r0 = params[2]
    
    coords = draw.circle(y0, x0, r0, shape=im2.shape)
    template = np.zeros_like(im2) #set all values to be zero
    template[coords] = 1
    
    mask_size = np.sum(template)
    cell_pixels_covered_by_mask = np.sum(template&im2)
    penalty_harshness = 10
    
    score = mask_size - penalty_harshness*cell_pixels_covered_by_mask
    score = score/area
        
    return -score

# Analyses

## Find cells on control

In [6]:
setup = {}
@genepattern.build_ui
def create_mask(control='analyses/MDA231_stopper_1_c3.tif',kernel_size=2,setup='setup'):
    beginning_of_time = timer()
    # Read image
    im_in = cv2.imread(control, cv2.IMREAD_GRAYSCALE)

    # Threshold. ==> These could be parameters
    # Set values equal to or above 20 to 0.
    # Set values below 20 to 255.
    th, im_th = cv2.threshold(im_in, 20, 255, cv2.THRESH_BINARY_INV)

    # Copy the thresholded image.
    im_floodfill = im_th.copy()

    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0,0), 255);

    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)

    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    io.imsave(fname='temp_output.png', arr=im_out)

    # im_out_inv = cv2.bitwise_not(im_out)


    # dilate the mask:
    k_size = kernel_size
    k_half = k_size/2
    kernel = np.ones((k_size,k_size),np.uint8)
    coords = draw.circle(k_half, k_half, k_half, shape=im_th.shape)
    kernel[coords] = 1 
    erosion = cv2.erode(im_out,kernel,iterations = 1)
    dilation = cv2.dilate(cv2.bitwise_not(erosion),kernel,iterations = 1)
    # cells_mask = cv2.bitwise_not(dilation)
    cells_mask = dilation/255
    
    setup['control_grayscale'] = im_in
    setup['mask'] = cells_mask


    io.imshow(cells_mask)
    plt.show()
    
    print("Note that a value of ~1 means that pixel belongs to the mask and it is rendered as white.")
    print("A value of 0 means it deos not belong the mask and it is rendered as black.")
    end_of_time = timer()
    spanned = end_of_time - beginning_of_time
    print(f"\nDone with this part of the workflow. Elapsed time: {humanfriendly.format_timespan(spanned)}.")
    return setup
    

UIBuilder(function_import='create_mask', name='create_mask', params=[{'name': 'control', 'label': 'control', '…

##  Find migration region

In [7]:
@genepattern.build_ui
def find_migration_region(setup='setup',finesse=20):
    beginning_of_time = timer()
    
    global im2
    im2 = setup['control_grayscale']>0.2
    im2 = im2.astype(int)
    
    maxy, maxx = im2.shape
    minx, miny = (0,0)
    maxr = min(maxx,maxy)/2

    x0 = im2.shape[1]/2
    y0 = im2.shape[0]/2
    r0 = min(im2.shape[1],im2.shape[0])/4
    
    xmid = im2.shape[1]/2
    ymid = im2.shape[0]/2
    rmid = min(xmid,ymid)

    coarse = finesse*1/3

    # do fit, here with leastsq model
    # minner = Minimizer(cost_obj, params)
    x_slice = slice(xmid-x0/4, xmid+x0/4, (x0/2)/coarse)
    y_slice = slice(ymid-x0/4, ymid+x0/4, (y0/2)/coarse)
    r_slice = slice(rmid-x0/4, rmid+x0/4, (r0/2)/finesse)
    rranges = (x_slice,y_slice, r_slice)
    print('About to perform optimization. This would take a few seconds to a few minutes.')

    resbrute = optimize.brute(cost, rranges,full_output=True)

    # result = minner.minimize(method='brute',ranges=rranges)
    # report_fit(result)
    print('############')
    method = 'scipy.brute'
    opt_params = resbrute[0]
    x_opt = opt_params[0]
    y_opt = opt_params[1]
    r_opt = opt_params[2]
    print("Optimal paramters are", [x_opt,y_opt,r_opt])
    f, ax = plt.subplots()
    circle = plt.Circle((x_opt, y_opt), r_opt, alpha = 0.5)
    ax.imshow(im2, cmap='gray', interpolation='nearest')
    ax.add_artist(circle)
    print('############')
    print(f'Method "{method}""\tobjective={cost([x_opt,y_opt,r_opt])}')
    print('############')
    plt.show()
    
    coords = draw.circle(y0, x0, r0, shape=im2.shape)
    template = np.zeros_like(im2) #set all values to be zero
    template[coords] = 1
    
    setup['im2'] = im2
    setup['opt_params'] = opt_params
    setup['x_opt'] = x_opt
    setup['y_opt'] = y_opt
    setup['r_opt'] = r_opt
    setup['circle'] = circle
    setup['coords'] = coords
    setup['template'] = template
    
    end_of_time = timer()
    spanned = end_of_time - beginning_of_time
    print(f"\nDone with this part of the workflow. Elapsed time: {humanfriendly.format_timespan(spanned)}.")
    
    return setup

UIBuilder(function_import='find_migration_region', name='find_migration_region', params=[{'name': 'setup', 'la…

## Quantify migration (load images & make final plot)

In [8]:
@genepattern.build_ui
def load_images(list_of_groups,folder='images',setup=setup,verbose=False):
    all_files = sorted(os.listdir(folder))
    
    filename = []
    condition = []
    percent_covered = []
    
    if isinstance(list_of_groups, str):
        list_of_groups = list_of_groups.split(', ')
    
    for category in list_of_groups:
        curr_files = [i for i in all_files if category in i]
        if verbose:
            print(category,curr_files)
        for image in curr_files:
            if verbose:
                print(f"\tWorking with {image}")
            current_filename = os.path.join(folder,image)
            im = io.imread(current_filename,as_gray=True)
            im01 = im>0
            im01 = im01.astype(int)
            if False:
                f, ax = plt.subplots()
                ax.imshow(im01, cmap='gray')
                circle = plt.Circle((setup['x_opt'], setup['y_opt']), setup['r_opt'], alpha = 0.5)
                ax.add_artist(circle)
                plt.show()
            
            # create the mask on top of this image
            coords = draw.circle(setup['y_opt'], setup['x_opt'], setup['r_opt'], shape=im01.shape)
            template = np.zeros_like(im01) #set all values to be zero
            template[coords] = 1
            cell_pixels_covered_by_mask = np.sum(template&im01)
#             print(100*cell_pixels_covered_by_mask/np.sum(template))
            filename.append(image)
            condition.append(category)
            percent_covered.append(100*cell_pixels_covered_by_mask/np.sum(template))
            
    df = pd.DataFrame({"condition": condition, "percent_covered": percent_covered, "filename" : filename})


    f, ax = plt.subplots(figsize=(16,9))
    ax=sns.barplot(x="condition", y="percent_covered", data=df, dodge=1, ax=ax, ci=None)
    ax=sns.stripplot(x="condition", y="percent_covered", data=df, ax=ax, linewidth=2, edgecolor='gray')
    add_stat_annotation(ax, data=df, x='condition', y='percent_covered',
                        boxPairList=[("untreated", "AGR2ab"),("untreated", "Taxol"),("untreated", "IgG")],
                        test='Mann-Whitney', textFormat='star', loc='inside', verbose=2)
    return 

UIBuilder(function_import='load_images', name='load_images', params=[{'name': 'list_of_groups', 'label': 'list…